<a href="https://colab.research.google.com/github/duynht/Greedy_InfoMax/blob/master/Gradient_isolated_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README
This notebook houses preliminary experiments for applying gradient-isolated training to Graph Neural Networks

# Keep Colab running in the background
Set a JavaScript interval to click on the connect button every 60 seconds. Open developer-settings (in your web-browser) with Ctrl+Shift+I then click on console tab and paste this to the console prompt. (for Mac press Option+Command+I)



```
<!-- function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000); -->
```
Little did I know Colab changed their selector id with this fancy update so we'll use the now code then. This time we have proper start and stop function. LOLOLOLOLOL

```
var startClickConnect = function startClickConnect(){
    var clickConnect = function clickConnect(){
        console.log("Connnect Clicked - Start");
        document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
        console.log("Connnect Clicked - End"); 
    };

    var intervalId = setInterval(clickConnect, 60000);

    var stopClickConnectHandler = function stopClickConnect() {
        console.log("Connnect Clicked Stopped - Start");
        clearInterval(intervalId);
        console.log("Connnect Clicked Stopped - End");
    };

    return stopClickConnectHandler;
};

var stopClickConnect = startClickConnect();
```


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!git clone https://github.com/duynht/Greedy_InfoMax.git
%cd /content/Greedy_InfoMax

Cloning into 'Greedy_InfoMax'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 376 (delta 16), reused 16 (delta 6), pack-reused 345
Receiving objects: 100% (376/376), 35.27 MiB | 19.46 MiB/s, done.
Resolving deltas: 100% (208/208), done.
/content/Greedy_InfoMax


# Graph package installation 

In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 92kB 4.1MB/s 
     |████████████████████████████████| 71kB 4.0MB/s 
     |████████████████████████████████| 92kB 6.4MB/s 
     |████████████████████████████████| 92kB 6.7MB/s 
     |████████████████████████████████| 133kB 11.3MB/s 
     |████████████████████████████████| 3.1MB 13.5MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 133kB 44.1MB/s 
     |████████████████████████████████| 235kB 45.5MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
  Created wheel for jsonpickle: filename=jsonpickle-0.9.6-cp36-none-any.whl size=29466 sha256=374445a579b9f7c20d4f8f6f4672fd0f2a1d137f59f5ed1e053af25645a48324
  Stored in directory: /root/.cache/pip/wheels/07/8b/41/8ce98f4737a9ff61b1bf2673f2abfe66a6a43ad6e91d2c9736
  Created wheel for munch: filename=munch-2.3.2-py2.py3-none-any.whl size=6613 sha256=7deab9b3ab159d5057bd9eac92303cfaa5ab7d827d644c97e3ef5c45d4aae35f
  Stored in directory: /root/.cache

# Import

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import torch_geometric.nn as pyg_nn

import torch_geometric.utils as pyg_utils

import networkx as nx
import numpy as npnpinpi
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import DataLoader
from torch_geometric.data import Data

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from PIL import Image

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [5]:
from GreedyInfoMax.vision.data import get_dataloader
from GreedyInfoMax.vision.arg_parser import arg_parser
from GreedyInfoMax.vision.models import load_vision_model, SmallModel
from GreedyInfoMax.utils import logger, utils

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import os
import os.path as osp

In [0]:
import pdb
# breakpoint() # not working for python 3.6

In [0]:
import sys
if '-f' in sys.argv:
  sys.argv.remove('-f')

# Utilities

In [0]:
def train(model, optimizer, save_dir=''):
  trainloader = DataLoader(stl10_train, batch_size=64, shuffle=True)
  testloader = DataLoader(stl10_test, batch_size=64, shuffle=True)
  
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model.to(device)

  best_acc = 0

  save_dir = osp.join("/content/drive/My Drive/Gradient-isolated GNN", save_dir)
  if not osp.exists(save_dir):
    os.makedirs(save_dir)
  checkpoint = "acc_{:.4f}--epochs_{}--batch_size_64--dim_9216_512_64_10--optim_Adam_lr_0.003--F_nll_loss.pt"

  epochs = 200
  for e in range(epochs):
    running_loss = 0
    train_acc = 0
    model.train()
    for batch in trainloader:
      optimizer.zero_grad()
      batch = batch.to(device)
      emb, logits = model(batch)
      labels = batch.y
      loss = model.loss(logits, labels)
      pred = logits.argmax(dim=1) 

      train_acc += pred.eq(labels).sum().item()
      running_loss += loss.item()

      loss.backward()
      optimizer.step()

    train_acc /= len(trainloader.dataset)
    running_loss /= len(trainloader.dataset)
    
    # cheating validation = test
    
    test_acc = test(model, testloader)
    if (test_acc >= best_acc):
      best_acc = test_acc

    if e % 5 == 0:
      print("Epoch {}/{}. Loss: {:.4f}. Train accuracy: {:.4f}. Test accuracy: {:.4f}".format(e+1, epochs, running_loss, train_acc, test_acc))

    if e == epochs - 1:
      print("Epoch {}/{}. Loss: {:.4f}. Train accuracy: {:.4f}. Test accuracy: {:.4f}".format(e+1, epochs, running_loss, train_acc, test_acc))
      torch.save(model.state_dict(), osp.join(save_dir,checkpoint.format(test_acc, e+1)))
      print("Best accuracy: {:.4f}. Last accuracy: {:.4f}".format(best_acc, test_acc))       


In [0]:
def test(model, testloader):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  accuracy = 0
  model.eval()
  for batch in testloader:
    with torch.no_grad():
      batch = batch.to(device)
      emb, logits = model(batch)
      pred = logits.argmax(dim=1)
      labels = batch.y
    
    accuracy += pred.eq(labels).sum().item()

  accuracy /= len(testloader.dataset)
  return accuracy


# Create a Custom Graph Dataset from STL10
Each image is divided into a $2\times2$ grid. Each patch of the grid is a node of the graph.


In [0]:
class GraphSTL10(InMemoryDataset):
  def __init__(self, root, split):
    # !python -m GreedyInfoMax.vision.modded_downstream_classification \
    # --model_path "/content/drive/My Drive/Greedy_InfoMax/logs/modded_vision_experiment_module_all" \
    # --model_num 5 \
    # --download_dataset
    self.opt = arg_parser.parse_args()
    self.opt.download_dataset=True
    self.opt.model_path = "/content/drive/My Drive/pretrainted-resnet50-infomax"
    self.opt.model_num = 5    
    
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    self.context_models = []
    for pid in range(4):
      context_model, _ = load_vision_model.load_model_and_optimizer(
                self.opt, reload_model=True, calc_loss=False, patch_idx=pid)
      context_model.module.switch_calc_loss(False)
      for param in context_model.parameters():
        param.requires_grad = False
      context_model.eval()
      context_model.to(self.device)
      self.context_models.append(context_model)
      
    self.dataloader = None

    _, _, trainloader, _, testloader, _ = get_dataloader.get_dataloader(self.opt)
    self.split = split
    if (split == 'train'):
      # self.dataset = datasets.STL10(root='/tmp/stl10_train', split='train', download=True)
      self.dataloader = trainloader
    if (split == 'test'):
      # self.dataset = datasets.STL10(root='/tmp/stl10_test', split='test', download=True)
       self.dataloader = testloader
    
    super(GraphSTL10, self).__init__(root)
    # self.num_classes = 10
    self.data, self.slices = torch.load(self.processed_paths[0])

  @property
  def raw_file_names(self):
    return []

  @property
  def processed_file_names(self):
    if (self.split == 'train'):
      return ['graphstl10_train.pt']
    if (self.split == 'test'):
      return ['graphstl10_test.pt']
    
    return []

  def download(self):
    pass

  def process(self):
    
    """Pytorch ResNet50
    # def crop(image,pc_height,pc_width):
    #   im_width, im_height = image.size
    #   for i in range(im_height//pc_height):
    #     for j in range(im_width//pc_width):
    #       box = (j*pc_width, i*pc_height, (j+1)*pc_width, (i+1)*pc_height)
    #       yield image.crop(box)

    # data_list = []

    # preprocess = transforms.Compose([transforms.Resize(256),
    #                                     transforms.CenterCrop(224),
    #                                     transforms.ToTensor(),
    #                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    # resnet50 = models.resnet50(pretrained=True)
    # resnet50.fc = nn.Identity()

    # for param in resnet50.parameters():
    #   param.requires_grad = False

    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # resnet50.to(device)

    # source_nodes = [i for i in range(0,4) for j in range(0,4)]
    # target_nodes = [j for i in range(0,4) for j in range(0,4)]
    # edge_index = torch.tensor([source_nodes, target_nodes],dtype = torch.long)

    # for imid, (image, label) in enumerate(self.dataset):
    #   # neighbors = np.arange(imid, imid+4)
    #   # mask = [1]*len(neighbors)
    #   im_height, im_width = image.size
    #   pc_height, pc_width = im_height//2, im_width//2
    #   node_features = []
    #   for pid,piece in enumerate(crop(image, pc_height, pc_width)):
    #     patch = Image.new('RGB', (pc_width, pc_height), 255)
    #     patch.paste(piece)
    #     patch = preprocess(patch)
    #     patch = patch.view(1,*patch.shape)
    #     patch = patch.to(device)
    #     patch = resnet50.forward(patch).to(torch.device('cpu'))
    #     node_features.append(torch.tensor(patch))
    """

    source_nodes = [i for i in range(0,4) for j in range(0,4)]
    target_nodes = [j for i in range(0,4) for j in range(0,4)]
    edge_index = torch.tensor([source_nodes, target_nodes],dtype = torch.long)

    data_list = []

    for bid, (imgs, labels) in enumerate(self.dataloader):
      batch_size, num_channels, img_height, img_width = imgs.shape
      if not(batch_size == self.opt.batch_size_multiGPU):
        continue
      patches = []
      node_features = []
      patches.append(imgs[:,:, :img_height//2, :img_width//2])
      patches.append(imgs[:,:, :img_height//2, img_width//2:])
      patches.append(imgs[:,:, img_height//2:, :img_width//2])
      patches.append(imgs[:,:, img_height//2:, img_width//2:])
      for pid in range(4):
        patch = patches[pid].to(self.device)
        patch = self.context_models[pid](patch,labels)[2]
        patch = patch.to(torch.device('cpu'))
        node_features.append(patch)

      node_features = torch.stack(node_features)
      node_features = node_features.transpose(0,1)
      node_features = node_features.view(*node_features.shape[:-3],-1)
      # pdb.set_trace()
      labels = labels.to(torch.device('cpu'))
      for gid in range(batch_size):  
        data = Data(x=node_features[gid], edge_index=edge_index.clone(), y=labels[gid].unsqueeze(0))
        data_list.append(data)
      # pdb.set_trace()
    
    data, slices = self.collate(data_list)
    torch.save((data, slices), self.processed_paths[0])
  

In [12]:
stl10_train = GraphSTL10('/graphstl10/',split='train')
stl10_test = GraphSTL10('/graphstl10/',split='test')

Contrasting against  16  negative samples
ModuleList(
  (0): ModuleList(
    (0): ResNet_Encoder(
      (model): Sequential(
        (Conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (layer 0): Sequential(
          (0): PreActBottleneckNoBN(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
            (shortcut): Sequential(
              (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
            )
          )
          (1): PreActBottleneckNoBN(
            (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
          )
          (2): PreActBottleneckNoBN(
            (conv1): Conv2d(256, 64, kernel_si

Extracting ./datasets/stl10_binary/stl10_binary.tar.gz to ./datasets/stl10_binary
Files already downloaded and verified
Files already downloaded and verified
Use (train+val) / test split
Processing...
Done!
Contrasting against  16  negative samples
ModuleList(
  (0): ModuleList(
    (0): ResNet_Encoder(
      (model): Sequential(
        (Conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (layer 0): Sequential(
          (0): PreActBottleneckNoBN(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
            (shortcut): Sequential(
              (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
            )
          )
          (1): PreActBottleneckNoBN(
            (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
            (conv2): Conv2d(64, 64, kernel_size

# Define custom MessagePassing [WIP]


In [0]:
class CustomConv(pyg_nn.MessagePassing):
  def __init__(self, in_channels, out_channels):
    super(CustomConv, self).__init__(aggr='add') #'add' aggregation
    self.lin = nn.Linear(in_channels, out_channels)
    self.lin_self = nn.Linear(in_channels, out_channels)

  def forward(self, x, edge_index):
    # x has shape [N, in_channels]
    # edge_index has shape [2, E]

    # Transform node feature matrix
    self_x = self.lin_self(x)
    # x = self.lin(x)

    return self_x + self.propagate(edge_index, size=(x.size(0), x.size(0)), x=self.lin(x))
  
  def message(self, x_i, x_j, edge_index, size):
    # Compute messages
    # x_j has shape [E, out_channels]
    # TODO:
    row, col = edge_index
    deg = pyg_utils.degree(row, size[0], dtype=x_j.dtype)
    deg_inv_sqrt = deg.pow(-0.5)
    norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

    return x_j

  def update(self, aggr_out):
    # aggr_out has shape [N, out_channels]
    return aggr_out

# Define the Graph Neural Network

## Graph Convolutional Network

In [0]:
class GCN(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(GCN, self).__init__()
        
    self.dropout = 0.25
    self.num_layers = 2
    self.hidden = [input_dim, 512, hidden_dim]
    # self.resnet = models.resnet50(pretrained=True)
    # self.resnet.fc = nn.Identity()

    self.convs = nn.ModuleList()
    self.lns = nn.ModuleList()

    for l in range(self.num_layers):
      self.convs.append(self.build_conv_model(self.hidden[l], self.hidden[l+1]))
      if (l + 1 < self.num_layers):
        self.lns.append(nn.LayerNorm(self.hidden[l+1]))

    # post-message-passing
    self.post_mp = nn.Sequential(
        nn.Linear(hidden_dim, hidden_dim), nn.Dropout(0.25),
        nn.Linear(hidden_dim, output_dim))

  def build_conv_model(self, input_dim, hidden_dim):
      return pyg_nn.GCNConv(input_dim, hidden_dim)
      
  def forward(self, data):
    x, edge_index = data.x, data.edge_index
    if data.num_node_features == 0:
      x = torch.ones(data.num_nodes, 1)

    for i in range(self.num_layers):
      x = self.convs[i](x, edge_index)
      emb = x
      x = F.relu(x)
      x = F.dropout(x, p=self.dropout, training = self.training)
      if not i == self.num_layers - 1:
        x = self.lns[i](x)

    x = pyg_nn.global_mean_pool(x, data.batch)
    # x = pyg_nn.global_add_pool(x, data.batch)

    x = self.post_mp(x)

    return emb, F.log_softmax(x, dim=1)

  def loss(self, pred, label):
    # Negative log-likelihood
    return F.nll_loss(pred, label)

In [37]:
model = GCN(input_dim=stl10_train.num_node_features, hidden_dim=64, output_dim = 10)
optimizer = optim.Adam(params=model.parameters(), lr=0.003)
train(model, optimizer)

Epoch 1/200. Loss: 0.0280. Train accuracy: 0.2959. Test accuracy: 0.4051
Epoch 6/200. Loss: 0.0140. Train accuracy: 0.6683. Test accuracy: 0.4735
Epoch 11/200. Loss: 0.0053. Train accuracy: 0.8840. Test accuracy: 0.4571
Epoch 16/200. Loss: 0.0028. Train accuracy: 0.9389. Test accuracy: 0.4579
Epoch 21/200. Loss: 0.0018. Train accuracy: 0.9619. Test accuracy: 0.4635
Epoch 26/200. Loss: 0.0012. Train accuracy: 0.9762. Test accuracy: 0.4650
Epoch 31/200. Loss: 0.0015. Train accuracy: 0.9710. Test accuracy: 0.4539
Epoch 36/200. Loss: 0.0011. Train accuracy: 0.9804. Test accuracy: 0.4622
Epoch 41/200. Loss: 0.0006. Train accuracy: 0.9872. Test accuracy: 0.4636
Epoch 46/200. Loss: 0.0008. Train accuracy: 0.9846. Test accuracy: 0.4606
Epoch 51/200. Loss: 0.0006. Train accuracy: 0.9910. Test accuracy: 0.4654
Epoch 56/200. Loss: 0.0014. Train accuracy: 0.9728. Test accuracy: 0.4710
Epoch 61/200. Loss: 0.0007. Train accuracy: 0.9882. Test accuracy: 0.4609
Epoch 66/200. Loss: 0.0007. Train accura

## Graph Attention Network


In [0]:
class GAT(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(GAT, self).__init__()
        
    self.dropout = 0.25
    self.num_layers = 2
    self.hidden = [input_dim, 512, hidden_dim]
    # self.resnet = models.resnet50(pretrained=True)
    # self.resnet.fc = nn.Identity()

    self.convs = nn.ModuleList()
    self.lns = nn.ModuleList()

    for l in range(self.num_layers):
      self.convs.append(self.build_conv_model(self.hidden[l], self.hidden[l+1]))
      if (l + 1 < self.num_layers):
        self.lns.append(nn.LayerNorm(self.hidden[l+1]))

    # post-message-passing
    self.post_mp = nn.Sequential(
        nn.Linear(hidden_dim, hidden_dim), nn.Dropout(0.25),
        nn.Linear(hidden_dim, output_dim))

  def build_conv_model(self, input_dim, hidden_dim):
      return pyg_nn.GATConv(input_dim, hidden_dim)
      
  def forward(self, data):
    x, edge_index = data.x, data.edge_index
    if data.num_node_features == 0:
      x = torch.ones(data.num_nodes, 1)

    for i in range(self.num_layers):
      x = self.convs[i](x, edge_index)
      emb = x
      x = F.relu(x)
      x = F.dropout(x, p=self.dropout, training = self.training)
      if not i == self.num_layers - 1:
        x = self.lns[i](x)

    x = pyg_nn.global_mean_pool(x, data.batch)
    # x = pyg_nn.global_add_pool(x, data.batch)

    x = self.post_mp(x)

    return emb, F.log_softmax(x, dim=1)

  def loss(self, pred, label):
    # Negative log-likelihood
    return F.nll_loss(pred, label)

In [33]:
model = GAT(input_dim=stl10_train.num_node_features, hidden_dim=64, output_dim = 10)
optimizer = optim.Adam(params=model.parameters(), lr=0.003)
train(model, optimizer, save_dir='ResNet50-GreedyInfoMax-GAT')

Epoch 1/200. Loss: 0.0301. Train accuracy: 0.2540. Test accuracy: 0.3399
Epoch 6/200. Loss: 0.0215. Train accuracy: 0.4914. Test accuracy: 0.3801
Epoch 11/200. Loss: 0.0132. Train accuracy: 0.6957. Test accuracy: 0.3698
Epoch 16/200. Loss: 0.0073. Train accuracy: 0.8347. Test accuracy: 0.3721
Epoch 21/200. Loss: 0.0047. Train accuracy: 0.8936. Test accuracy: 0.3751
Epoch 26/200. Loss: 0.0037. Train accuracy: 0.9229. Test accuracy: 0.3655
Epoch 31/200. Loss: 0.0032. Train accuracy: 0.9315. Test accuracy: 0.3656
Epoch 36/200. Loss: 0.0017. Train accuracy: 0.9649. Test accuracy: 0.3668
Epoch 41/200. Loss: 0.0020. Train accuracy: 0.9595. Test accuracy: 0.3636
Epoch 46/200. Loss: 0.0016. Train accuracy: 0.9675. Test accuracy: 0.3690
Epoch 51/200. Loss: 0.0015. Train accuracy: 0.9698. Test accuracy: 0.3595
Epoch 56/200. Loss: 0.0017. Train accuracy: 0.9661. Test accuracy: 0.3628
Epoch 61/200. Loss: 0.0012. Train accuracy: 0.9744. Test accuracy: 0.3589
Epoch 66/200. Loss: 0.0019. Train accura

## Attention-based Graph Neural Network

In [0]:
class AGNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(AGNN, self).__init__()
        
    self.dropout = 0.25
    self.num_layers = 2
    self.hidden = [input_dim, 512, hidden_dim]
    # self.resnet = models.resnet50(pretrained=True)
    # self.resnet.fc = nn.Identity()

    self.convs = nn.ModuleList()
    self.lns = nn.ModuleList()
    self.attention = pyg_nn.AGNNConv()

    for l in range(self.num_layers):
      self.convs.append(self.build_conv_model(self.hidden[l], self.hidden[l+1]))
      if (l + 1 < self.num_layers):
        self.lns.append(nn.LayerNorm(self.hidden[l+1]))

    # post-message-passing
    self.post_mp = nn.Sequential(
        nn.Linear(hidden_dim, hidden_dim), nn.Dropout(0.25),
        nn.Linear(hidden_dim, output_dim))

  def build_conv_model(self, input_dim, hidden_dim):
      return nn.Linear(input_dim, hidden_dim)
      
  def forward(self, data):
    x, edge_index = data.x, data.edge_index
    if data.num_node_features == 0:
      x = torch.ones(data.num_nodes, 1)

    for i in range(self.num_layers):
      x = self.attention(self.convs[i](x), edge_index)
      emb = x
      x = F.relu(x)
      x = F.dropout(x, p=self.dropout, training = self.training)
      if not i == self.num_layers - 1:
        x = self.lns[i](x)

    x = pyg_nn.global_mean_pool(x, data.batch)
    # x = pyg_nn.global_add_pool(x, data.batch)

    x = self.post_mp(x)

    return emb, F.log_softmax(x, dim=1)

  def loss(self, pred, label):
    # Negative log-likelihood
    return F.nll_loss(pred, label)

In [35]:
model = AGNN(input_dim=stl10_train.num_node_features, hidden_dim=64, output_dim = 10)
optimizer = optim.Adam(params=model.parameters(), lr=0.003)
train(model, optimizer, save_dir="ResNet50-GreedyInfoMax-AGNN")

Epoch 1/200. Loss: 0.0281. Train accuracy: 0.2849. Test accuracy: 0.3760
Epoch 6/200. Loss: 0.0129. Train accuracy: 0.6963. Test accuracy: 0.4730
Epoch 11/200. Loss: 0.0046. Train accuracy: 0.8992. Test accuracy: 0.4672
Epoch 16/200. Loss: 0.0018. Train accuracy: 0.9607. Test accuracy: 0.4635
Epoch 21/200. Loss: 0.0015. Train accuracy: 0.9692. Test accuracy: 0.4541
Epoch 26/200. Loss: 0.0015. Train accuracy: 0.9722. Test accuracy: 0.4689
Epoch 31/200. Loss: 0.0012. Train accuracy: 0.9748. Test accuracy: 0.4641
Epoch 36/200. Loss: 0.0014. Train accuracy: 0.9740. Test accuracy: 0.4590
Epoch 41/200. Loss: 0.0007. Train accuracy: 0.9856. Test accuracy: 0.4625
Epoch 46/200. Loss: 0.0005. Train accuracy: 0.9904. Test accuracy: 0.4695
Epoch 51/200. Loss: 0.0015. Train accuracy: 0.9698. Test accuracy: 0.4482
Epoch 56/200. Loss: 0.0008. Train accuracy: 0.9842. Test accuracy: 0.4582
Epoch 61/200. Loss: 0.0007. Train accuracy: 0.9844. Test accuracy: 0.4557
Epoch 66/200. Loss: 0.0007. Train accura

In [0]:
%debug